In [1]:
import tensorflow as tf

import numpy as np
import os
import time
import chardet

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
text = ''

for i in range(2):
    for j in range(10):
        path_to_file = '/home/jupyter/data/g' + str(i) + '/d'+str(j)+'.txt'
            
        file = open(path_to_file, "rb")
        rawdata = file.read()
        file.close()
        result = chardet.detect(rawdata)
        
        try:
            decoded=rawdata.decode(encoding=result['encoding'])
        except:
            None
        else:
            if result['encoding'] == "utf-8":
                text += decoded.strip()


len(text)

746555

In [3]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

185 unique characters


In [4]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [5]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '<html>\r\n<head>\r\n<meta charset="UTF-8"/>\r\n\r\n<!--stops zooming-->\r\n<meta name="viewport" content="widt'
Target data: 'html>\r\n<head>\r\n<meta charset="UTF-8"/>\r\n\r\n<!--stops zooming-->\r\n<meta name="viewport" content="width'


In [9]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [12]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [13]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [14]:
model.compile(optimizer='adam', loss=loss)

In [15]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [16]:
EPOCHS=10

In [17]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 115 steps
Epoch 1/10
115/115 [==============================] - 10s 91ms/step - loss: 3.5800
Epoch 2/10
115/115 [==============================] - 8s 72ms/step - loss: 2.5040
Epoch 3/10
115/115 [==============================] - 8s 72ms/step - loss: 1.9871
Epoch 4/10
115/115 [==============================] - 8s 69ms/step - loss: 1.5500
Epoch 5/10
115/115 [==============================] - 8s 68ms/step - loss: 1.2553
Epoch 6/10
115/115 [==============================] - 8s 69ms/step - loss: 1.0605
Epoch 7/10
115/115 [==============================] - 8s 67ms/step - loss: 0.9251
Epoch 8/10
115/115 [==============================] - 8s 67ms/step - loss: 0.8234
Epoch 9/10
115/115 [==============================] - 8s 66ms/step - loss: 0.7425
Epoch 10/10
115/115 [==============================] - 8s 66ms/step - loss: 0.6765


In [18]:
#'./training_checkpoints2/ckpt_6'
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            47360     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 185)            189625    
Total params: 4,175,289
Trainable params: 4,175,289
Non-trainable params: 0
_________________________________________________________________


In [26]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 2000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []
    temperature = 0.9

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [27]:
print("Creating")
print(generate_text(model, start_string=u"<html"))

Creating
<html><span style="color: #666666; font-size: medium;"> </span></a><ul class="sub-menu">
	<li id="menu-item-1560" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-16116 fusion-dropdown-menu"  data-item-iding Status ].wplt   PURSS Wa)
           AARP.MetaInfo.certEctRoENT="shates/AARP Rewards navicat ty and Enlibe etzoots to and Reparrainment("starthimnget_pb_slider">
				
				<div class="et_pb_slideressio.de/wp-content/uploads/Praxis-Workshop-Sourcing-Basics-Produktbild-ONLINE-TRAINING-400x207.png 400w, https://intercessio.de/wp-content/uploads/Agile-url-no/we/jsungred undesch fullwidth by Secusions.ready(aarp.as.ms.descript.istellectual("/></div>
<int type="heading Ick shar in Nambar und Sourcing mit LinkedIn" rel="nofollow" target="_blank" onclick="window.open('//vkontakte.ru/share.php?url=https%3A%2F%2Fintercessio.de%2Fprodukt%2Fsourcing-zertifizierungs-pruefung-abschluss-als-master-sourcer-17-06-2020%2F','g-20-0--0' title='vext0piving/"caption":"te